In [39]:
import pandas as pd
import numpy as np
SEED = 42 

df_train = pd.read_csv("train.csv", index_col=0)
df_train["coeff0"] = df_train["loan_amount"]/df_train["number_of_owner_occupied_units"]
df_train["coeff1"] = df_train["minority_population"]/df_train["population"]

In [31]:
df_train.columns

Index(['applicant_ethnicity', 'income', 'applicant_race_1', 'applicant_race_2',
       'applicant_race_3', 'applicant_race_4', 'applicant_race_5',
       'applicant_sex', 'co_applicant_ethnicity', 'co_applicant_race_1',
       'co_applicant_race_2', 'co_applicant_race_3', 'co_applicant_race_4',
       'co_applicant_race_5', 'co_applicant_sex', 'loan_purpose', 'loan_type',
       'msamd', 'preapproval', 'property_type', 'purchaser_type',
       'hud_median_family_income', 'loan_amount',
       'number_of_1_to_4_family_units', 'number_of_owner_occupied_units',
       'minority_population', 'population', 'target', 'coeff0', 'coeff1'],
      dtype='object')

In [32]:
def one_hot_encode_feature(dataframe, feature_name):
    # Perform one-hot encoding on the specified feature
    encoded_df = pd.get_dummies(dataframe, columns=[feature_name], prefix=[feature_name])

    return encoded_df
one_hot_encode_feature(df_train, "loan_purpose").columns

Index(['applicant_ethnicity', 'income', 'applicant_race_1', 'applicant_race_2',
       'applicant_race_3', 'applicant_race_4', 'applicant_race_5',
       'applicant_sex', 'co_applicant_ethnicity', 'co_applicant_race_1',
       'co_applicant_race_2', 'co_applicant_race_3', 'co_applicant_race_4',
       'co_applicant_race_5', 'co_applicant_sex', 'loan_type', 'msamd',
       'preapproval', 'property_type', 'purchaser_type',
       'hud_median_family_income', 'loan_amount',
       'number_of_1_to_4_family_units', 'number_of_owner_occupied_units',
       'minority_population', 'population', 'target', 'coeff0', 'coeff1',
       'loan_purpose_1', 'loan_purpose_2', 'loan_purpose_4', 'loan_purpose_5',
       'loan_purpose_31', 'loan_purpose_32'],
      dtype='object')

In [24]:
arr = np.zeros(6)
for index, row in df_train.iterrows():
    if(row["target"] == 1):
        if(row["loan_purpose"] == 1):
            arr[0]+=1
        elif(row["loan_purpose"] == 32):
            arr[1]+=1
        elif(row["loan_purpose"] == 31):
            arr[2]+=1
        elif(row["loan_purpose"] == 2):
            arr[3]+=1
        elif(row["loan_purpose"] == 4):
            arr[4]+=1
        else:
            arr[5]+=1
    

In [43]:
df_train['loan_purpose'].value_counts()

loan_purpose
1     549339
32    248143
31    115804
2      98090
4      92484
5       3751
Name: count, dtype: int64

In [46]:
result = df_train.groupby('loan_purpose')['target'].mean() * 100
result

loan_purpose
1     52.597212
2     40.475074
4     37.670300
5      3.252466
31    43.245484
32    42.838605
Name: target, dtype: float64

In [14]:
df_train["loan_purpose"].value_counts()

loan_purpose
1     549339
32    248143
31    115804
2      98090
4      92484
5       3751
Name: count, dtype: int64

In [12]:
df_train[['applicant_ethnicity', 'preapproval', 'coeff0', 'coeff1', 'target']].corr()

,applicant_ethnicity,preapproval,coeff0,coeff1,target
applicant_ethnicity,1.000000,0.026788,0.011521,-0.000845,-0.178927
preapproval,0.026788,1.000000,0.000062,0.000791,-0.063208
coeff0,0.011521,0.000062,1.000000,0.016457,0.004289
coeff1,-0.000845,0.000791,0.016457,1.000000,-0.004400
target,-0.178927,-0.063208,0.004289,-0.004400,1.000000


In [5]:
df_train['minority_population'].value_counts()

minority_population
0.00     10675
21.00     2521
29.36     2407
30.72     2402
53.67     2394
         ...  
80.06        2
84.27        1
35.01        1
82.47        1
47.80        1
Name: count, Length: 3847, dtype: int64

In [7]:
df_train['minority_population'].quantile(0.95)


91.64

In [18]:
df_train = df_train.astype({"applicant_ethnicity": "float64"})
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1107611 entries, 0 to 1107610
Data columns (total 28 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   applicant_ethnicity             1107611 non-null  float64
 1   income                          984345 non-null   float64
 2   applicant_race_1                1107376 non-null  float64
 3   applicant_race_2                35442 non-null    float64
 4   applicant_race_3                3369 non-null     float64
 5   applicant_race_4                374 non-null      float64
 6   applicant_race_5                106 non-null      float64
 7   applicant_sex                   1107611 non-null  int64  
 8   co_applicant_ethnicity          1107611 non-null  int64  
 9   co_applicant_race_1             1107523 non-null  float64
 10  co_applicant_race_2             11395 non-null    float64
 11  co_applicant_race_3             999 non-null      float64
 12  co_ap

In [25]:
import pandas as pd

# Define the dataframe
data = {'feature1': [1, 2, 3, 4, 5],
        'feature2': [0.1, 0.5, 0.3, 0.8, 0.2],
        'target': [0, 1, 1, 0, 1]}

df = pd.DataFrame(data)


import lightgbm as lgb

def convert_to_lgb_dataset(df):
    # Separate features and target variable
    features = df.drop('target', axis=1)
    target = df['target']
    
    # Create LightGBM Dataset
    lgb_dataset = lgb.Dataset(data=features, label=target)
    
    return lgb_dataset

# Example usage
# Assuming you have a pandas DataFrame called 'df'

lgb_dataset = convert_to_lgb_dataset(df_train)
lgb_dataset.construct()
lgb_dataset.num_feature()

27

In [ ]:
cols = df_train.columns
for i in cols: 
    print(i)
    print(len(set(df_train[i].values)))
    print(set(df_train[i].values))


